In [1]:
import           Control.Monad.Trans.Reader
import qualified Data.Map.Strict as Map

data Expr a
    = Var Int
    | App (Expr a) (Expr a)
    | Lam (Expr a)
    | Lit a
    | Let String (Expr a) (Expr a)
    | Bnd String
    deriving (Show)

data Env a = Env
    { lets :: Map.Map String a
    , vars :: [a]
    } deriving (Show)

newtype Eval a = Eval { runEval :: Reader (Env a) a }

In [2]:
push :: a -> Env a -> Env a
push v (Env lets vars) = Env lets (v:vars)

bind :: String -> a -> Env a -> Env a
bind n v (Env lets vars) = Env (Map.insert n v lets) vars

eval :: Show a => Expr a -> Eval a
eval expr = case expr of
    Var i -> Eval $ do
        vars <- vars <$> ask
        pure $ vars !! i
    App f x -> Eval $ do
        x' <- runEval $ eval x
        local (push x') (runEval (eval f))
    Lam f -> eval f
    Lit a -> Eval $ pure a
    Let n v e -> Eval $ do
        v' <- runEval $ eval v
        local (bind n v') (runEval (eval e))
    Bnd n -> Eval $ do
        lets <- lets <$> ask
        pure $ lets Map.! n

k = (Lam (Lam (Var 1)))
i = (Lam (Var 0))
s = (Lam (Lam (Lam (App (App (Var 2) (Var 0)) (App (Var 1) (Var 0))))))

i' = App (App s k) k


e = App (App k (Lit 5)) (Lit 7)

l = Let "x" (Lit 1) (Let "x" (Lit 2) (Bnd "x"))


runReader (runEval (eval l)) (Env Map.empty [])
runReader (runEval (eval e)) (Env Map.empty [])

2

7

In [3]:
subY = Lam (App (Var 1) (App (Var 0) (Var 0)))
y = Lam (App subY subY) 